In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
##random sampling

In [ ]:
#creates sample table of specified size
def create_sample_table (nrow, data):
    sample_table = np.empty(shape=(nrow, data.shape[1]))
    for c in range (0,sample_table.shape[1]):
        sample_table[:,c] = np.random.choice (data[:,c], nrow, replace = False)
    return sample_table

In [ ]:
#get the MSE for one dataset, aggregate, and sample table size
def MSE_one_example (sample_table_nrows, data, aggregate, quantile):
    sample_table = create_sample_table (sample_table_nrows, data)
    MSE = 0
    for c in range (0,sample_table.shape[1]):
        if (quantile != 0):
            actual = aggregate(data[:,c], quantile)
            sample = aggregate(sample_table[:,c], quantile)
            MSE += (actual-sample)**2
        else:
            actual = aggregate(data[:,c])
            sample = aggregate(sample_table[:,c])
            MSE += (actual-sample)**2 
    MSE = MSE/sample_table.shape[1]
    return MSE

In [ ]:
#prints MSE results for different sample table sizes
def get_results (sample_table_sizes, data, aggregate, i, dataset_size):    
    aggregate_mse = []
    for sample_table_ratio in sample_table_sizes:
        aggregate_mse.append(MSE_one_example (round(sample_table_ratio*data.shape[0]), data, aggregate, quantile = i))
    print ("***New Example***")
    print ("dataset size: ")
    print(dataset_size)
    print ("aggregate: ")
    print(aggregate)
    print ("quantile: ")
    print (i)
    print ("sample table sizes: ")
    print(sample_table_sizes)
    print ("MSE values: ")
    print(aggregate_mse)
    plt.plot (sample_table_sizes, aggregate_mse)
    plt.pause(0.05)
    plt.show()
    print ("********") 

In [ ]:
#takes in list of dataset sizes, list of aggregates, list of sample table sizes and plots MSE vs. sample table
#size for each dataset
#***dataset is ~N(0,1)
#@param dataset_sizes (list of tuples)
#@param aggregates (list of numpy aggregates on one column)
#@param sample_table_sizes (list of numbers representing ratios of sample table size to data table size)
#@param outliers (whether to do simple outliers test)
def eval_random_sampling_MSE (dataset_sizes, aggregates, sample_table_sizes, outliers, stdev):
    for dataset_size in dataset_sizes:
        data = np.random.normal(loc = 0, scale = stdev, size=dataset_size)
        if (outliers):
            for c in range (0, data.shape[1]):
                for r in range (0, data.shape[0]):
                    if (r%200 == 0):
                        data [r, c] += 100   
        for aggregate in aggregates:
            if (aggregate == np.quantile):
                for i in [.1, .25, .5, .75]:
                    get_results (sample_table_sizes, data, aggregate, i, dataset_size)
            else:
                get_results (sample_table_sizes, data, aggregate, 0, dataset_size)
    plt.show()

In [ ]:
#Test on some examples
dataset_sizes = [(100, 100), (250, 250), (500, 500), (1000, 1000), (2500, 2500), (5000, 5000)]
aggregates = [np.mean, np.std, np.amax, np.quantile]
sample_table_sizes = [.01, .05, .1, .25, .5, .75, .9]

In [ ]:
eval_random_sampling_MSE (dataset_sizes, aggregates, sample_table_sizes, False, 1)

In [ ]:
#Test with outliers.  Notice much more variation for smaaller sample table sizes
eval_random_sampling_MSE (dataset_sizes, aggregates, sample_table_sizes, True, 1)

In [ ]:
#Test with high stdev - notice error scales proportionally
eval_random_sampling_MSE (dataset_sizes, aggregates, sample_table_sizes, True, 100)

In [ ]:
#Test on categorical dataset.  Cat1 = 0:.25, Cat2 = .25:.5, Cat3 = . .5:.75, Cat4 =.75:1 

#Save results
sum_one_category_MSE = []

data = np.random.normal(loc = 0, scale = 1, size=(1000,1000))
sample_table_sizes = [50, 100, 250, 500, 750, 900]

for i in range(0,len(sample_table_sizes)):
    sample_table_size = sample_table_sizes[i]
    sample_table = create_sample_table (sample_table_size, data)
    MSE = 0
    for c in range (0,sample_table.shape[1]):
        num_Cat1_sample = 0
        num_Cat1_actual = 0
        for i in range (0, sample_table.shape[0]):
            if (sample_table[i,c] <= .25):
                num_Cat1_sample += 1
        for i in range (0, data.shape[0]):
            if (data[i,c] <= .25):
                num_Cat1_actual += 1
        sample = num_Cat1_sample/sample_table.shape[0]
        actual = num_Cat1_actual/data.shape[0]
        MSE += (actual-sample)**2
    MSE = MSE/sample_table.shape[1]
    sum_one_category_MSE.append(MSE)
   
plt.plot (sample_table_sizes, sum_one_category_MSE)    

In [ ]:
#Second exploration: a quantile approximation algorithm 
#https://pdfs.semanticscholar.org/3593/8dc843cb7ce95be5007ec40e3967ab6bfae8.pdf
def Agrawal_Swami_generic (X, k, p):
    H = []
    tau = p*len(X)
    for x in X:
        contains_x = False
        for e_i in H:
            if (e_i[0] == x):
                e_i[1] += 1
                contains_x = True
                break
        if (contains_x == False):     
            if (len(H) < k):
                if (len(H)==0):
                    H.insert (0, [x,1])
                else:
                    #insert (x,1) maintaining sorted order
                    append = True
                    for i in (range(0,len(H))):
                        if (H[i][0] > x):
                            H.insert (i, [x, 1])
                            append = False
                            break      
                    if (append):
                        H.append ([x,1])
            elif (x < H[0][0]):
                N_1_H = np.sum([entry[1] for entry in H])
                if (N_1_H < tau):
                    H[len(H)-2][1] = H[len(H)-2][1] + H[len(H)-1][1]
                    del H[len(H)-1]
                    #insert (x,1)
                    append = True
                    for i in (range(0,len(H))):
                        if (H[i][0] > x):
                            H.insert (i, [x, 1])
                            append = False
                            break
                    if (append):
                        H.append ([x,1])
            else:
                for i in range(0,len(H)-1):
                    if (H[len(H)-1-i][0]<x):
                        H[len(H)-1-i][1] += 1
                        break
        N_2_H = np.sum([entry[1] for entry in H[1:]])
        if(N_2_H>=tau):
            del H[0]
    return H[0][0]

In [ ]:
#test on some examples with relative error
X = np.random.randint (0, 1000, (1,100000))[0]
k_sizes = [10, 50, 100, 250, 500]
p_vals = [.5]
for p in p_vals:
    relative_errors = []
    for k in k_sizes:
        approximate_median = Agrawal_Swami_generic(X, k, p)
        median = np.median(X)
        relative_error = abs(approximate_median-median)/approximate_median
        relative_errors.append(relative_error)
    print ("p_val: ")
    print (p)
    plt.plot(k_sizes, relative_errors)
    plt.pause(0.05)
plt.show()

In [ ]:
#Another quantile summary: http://infolab.stanford.edu/~datar/courses/cs361a/papers/quantiles.pdf
#this is an e-quantile summary
X = np.random.randint (0, 1000, (1,100000))[0]

#an ordered sequence of tuples which correspond to a subset of the observations seen thus far
summary_structure = []


In [ ]:
#Pseudocode
def GK_summary (data, e, S):
    for n in range (0,len(data)):
        if (n%(1/2e) == 0):
            compress(S)
        insert(data[n]) 

In [ ]:
def quantile (q, e, data):
    n = len(data) 
    #load t_i into T
    s=0
    S = tree with just root node R
    GK_summary(data, e, S):
    r = ceiling (q*n)
    find i s/t both (r-r_min(v_i))<=e*n and (r_max(v_i)-r)<=e*n
    return v_i

In [ ]:
def insert (v)
    find the smallest i s/t v_(i-1) <= v < v_i
    insert tuple (v, 1, floor(2*e*n)) between t_(i-1) and t_i
    if (v = min or max seen):
        insert (v, 1, 0)
    else:
        
    s +=1

In [ ]:
def delete (v_i, v_(i+1)):
    replace (v_i, g_i, delta_i) and (v_(i+1),g_(i+), delta_(i+1) with 
                                     new (v_(i+1),g_i + g_(i+), delta_(i+1))
    s=s-1

In [ ]:
def compress(delta_t, e, n, ):
    for i in range (s-2,0):
        if (band(delta_i,2*e*n)<= band (delta_(i+1), 2*e*n) and (g_i_star + g_(i+1) + delta _(i_1))<2*e*n )
            delete all descendents of t_i and t_i